In [137]:
%matplotlib notebook

import pandas as pd
pd.set_option('display.max_colwidth', -1)
import networkx as nx

import itertools as it
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import sys
from importlib import reload
sys.path.append('./chinese-whispers-python')
sys.path.append('./')
import solution
from pprint import pprint
from pymystem3 import Mystem
morph = Mystem()

In [138]:
reload(solution)
model = None
try:
    model = sg.model
except:
    pass
sg = solution.SenseGraph(model=model, debug=False)

In [140]:
tags=('NOUN', 'VERB', 'ADJ', 'ADV', 'ADV', 'ADJ', 'DET', 'ADJ', 'SCONJ', 'INTJ', 'X', 'NUM', 'PART', 'ADP', 'PRON')
res = []
for t in tags:
    try:
        sg.N = 10
        res = sg.top_k('рыболовство_' + t)
        break
    except:
        pass
res

['рыболовство_NOUN',
 'марикультура_NOUN',
 'рыбоводство_NOUN',
 'главрыбвод_NOUN',
 'водный::биоресурсы_NOUN',
 'любительский::рыболовство_NOUN',
 'рыбный::ловля_NOUN',
 'оленеводство_NOUN',
 'агатополь_NOUN',
 'соболевание_NOUN']

### Train Test dataset overview

In [134]:
dataset_fpath = "data/main/active-dict/train.csv"
train_df = pd.read_csv(dataset_fpath, sep='\t', dtype={'gold_sense_id': str, 'predict_sense_id': str})

dataset_fpath_additional = "data/additional/active-rutenten/train.csv"
test_df = pd.read_csv(dataset_fpath_additional, sep='\t', dtype={'gold_sense_id': str, 'predict_sense_id': str})

In [135]:
train_words = train_df['word'].unique()
test_words = test_df['word'].unique()
diff_test_train = set(test_words) - set(train_words)
print('Train[{}]: {}\n\nTest[{}]: {}\n\nTest-Train[{}]: {}'.
      format(len(train_words), sorted(train_words), len(test_words), sorted(test_words), 
             len(diff_test_train), sorted(diff_test_train)))

Train[85]: ['дар', 'двигатель', 'двойник', 'дворец', 'девятка', 'дедушка', 'дежурная', 'дежурный', 'декабрист', 'декрет', 'дело', 'демобилизация', 'демократ', 'демонстрация', 'дерево', 'держава', 'дерзость', 'десятка', 'десяток', 'деятель', 'диалог', 'диаметр', 'диплом', 'директор', 'диск', 'дичь', 'длина', 'доброволец', 'добыча', 'доказательство', 'доктор', 'долгота', 'доля', 'дом', 'дорога', 'достижение', 'древесина', 'дупло', 'дура', 'дух', 'дым', 'дымка', 'дыхание', 'дьявол', 'евро', 'езда', 'жаворонок', 'жало', 'жертва', 'жестокость', 'жидкость', 'жила', 'жилец', 'жир', 'жребий', 'заведение', 'завещание', 'зависимость', 'заголовок', 'заготовка', 'задание', 'задача', 'задержка', 'зажигалка', 'закон', 'закрытие', 'заложник', 'замена', 'западня', 'запятая', 'застой', 'затея', 'затишье', 'затмение', 'затруднение', 'захоронение', 'звезда', 'звон', 'зеркало', 'зло', 'злоупотребление', 'знак', 'знамя', 'значение', 'зонт']

Test[20]: ['альбом', 'анатомия', 'базар', 'балет', 'беда', 'бездн

### Text analysis of train data

In [136]:
import re
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN, ORANGE
def markup_texts(df, word = None):
    if word:
        df = df[df['word'] == word]
    a_word = None
    a_sense = None
    for word, text, span, golder_sense in df[['word', 'context', 'positions', 'gold_sense_id']].values:
        if a_word != word:
            sense_id = 0
            a_sense = golder_sense
            a_word = word
            print('\n', a_word.upper())
        if golder_sense != a_sense:
            a_sense = golder_sense

        try:
            spans = tuple(map(int, re.split(r'[,-]', span)))
        except:
            print(text)
            continue
        spans = [[spans[i], spans[i+1], a_sense] for i in range(0, len(spans), 2)]
        show_box_markup(text, spans)
    
markup_texts(train_df, word='дар')


 ДАР


BoxMarkup('Отвергнуть щедрый дар', [Span(18, 22, '1')])

BoxMarkup('покупать преданность дарами и наградами', [Span(21, 28, '1')])

BoxMarkup('Вот яд – последний дар моей Изоры', [Span(19, 23, '1')])

BoxMarkup('Основная функция корильных песен – повеселить участников свадьбы и побудить их к дарам',
          [Span(81, 87, '1')])

...

BoxMarkup('В конце была приписка, что священник приехал, исповедовал больную глухою исповедью и приобщил запасными дарами',
          [Span(104, 111, '2.3')])

BoxMarkup('Он предупредил меня, что мы заедем в костел за святыми дарами для причащения умирающего и что с человеком, который везет святые дары, нельзя разговаривать',
          [Span(55, 62, '2.3'), Span(128, 133, '2.3')])

BoxMarkup('Как я могу служить обедню? У меня ведь нет ни даров, ни епитрахили',
          [Span(46, 52, '2.3')])

BoxMarkup('Поднимешь глаза – на амвоне стоит священник со святыми Дарами, и чудится, скорбный свет идет от чаши',
          [Span(55, 62, '2.3')])

BoxMarkup('Джованни хотелось, чтобы с ним повторилось чудо, о котором говорил Савонарола, – чтобы огненные лучи, выйдя из чаши со Святыми Дарами, выжгли в теле его, как раскаленное железо, крестные раны',
          [Span(127, 134, '2.3')])

### Features, select weights

In [129]:
features_fpath = "data/main/active-dict/train.features.csv"

tf = pd.read_csv(features_fpath, sep='\t', dtype={'gold_sense_id': str, 'predict_sense_id': str})
print(tf.shape)
tf.head(1)

(2073, 11)


,context_id,word,gold_sense_id,predict_sense_id,positions,context,context_wo,contextmorph,wordpos,feature_cos,feature_prob
0,1,дар,1,0,18-22,Отвергнуть щедрый дар,Отвергнуть щедрый,отвергать_VERB щедрый_ADJ,дар_NOUN,"[0.7531211, 0.31753352, 0.43184838, 0.34359467, 0.3427364, 0.3976531, 0.42096314, 0.3693876, 0.25753662]","[0.6209667614026362, 0.5586842328199068, 0.579046030753896, 0.5658078843067446, 0.5641240613114294, 0.5739783457251805, 0.5811552387437565, 0.5701889208658522, 0.5489347948982604]"


In [130]:
tf['feature_cos'] = tf['feature_cos'].apply(lambda _ : np.fromstring(_[1:-1], dtype=float, sep=', '))
tf['feature_prob'] = tf['feature_prob'].apply(lambda _ : np.fromstring(_[1:-1], dtype=float, sep=', '))

In [ ]:
from tqdm import tqdm_notebook as tqdm
def evaluate(tf, w, is_per = False):
    from sklearn.metrics import adjusted_rand_score
    tf['pred_id'] = (tf['feature_cos'].apply(lambda _ : _ * w)).apply(np.argmax)
    per_word = tf.groupby('word').aggregate(
                lambda f: adjusted_rand_score(
                    f['gold_sense_id'], f['pred_id']))
    per_word_ari = per_word['pred_id']
    if is_per:
        for word, ari in zip(per_word.index, per_word_ari):
            print('{:<20} {:+.4f}'.format(word, ari))
    return np.mean(per_word_ari)

w_list = []
res_list = []
for i in tqdm(range(2000), total=3000):
    w = np.random.uniform(0, 1, 9)
    res = evaluate(tf, w, False)
    w_list.append(w)
    res_list.append(res)

print('\n', max(res_list))
"[{}]".format(','.join(map(str, w_list[np.argmax(res_list)])))